<a href="https://colab.research.google.com/github/ecloguehwang/HSS/blob/master/1%EA%B0%9C_%ED%95%A9%EB%B6%88%ED%8C%8C%EC%9D%BC_2%EA%B0%9C%EC%8B%9C%ED%8A%B8_%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%8B%A0%EB%AA%A92021%ED%95%99%EB%85%84%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#1개 합불파일에 2개(수시, 정시)의 시트가 있을때 새로운 합불파일로 만들기

In [ ]:
#엑셀 파일 불러오기

##라이브러리
import pandas as pd


##엑셀파일 불러오기
#df_jsi = pd.read_excel(f'{path}habbul_2020sinmok.csv', encoding = 'cp949')
path = '/content/drive/MyDrive/python/sinmok/'
df = pd.read_excel(f'{path}habbul_2020sinmok.xlsx')

# 데이터 확인
df.head(3)

In [ ]:
#1개 파일에 2개의 시트(수시, 정시)를 하나로 합치기 - 2020sinmok.xlsx으로 테스트

import pandas as pd

# 엑셀 파일 경로 설정
path = '/content/drive/MyDrive/python/sinmok/'
file_path = f'{path}habbul_2020sinmok.xlsx'

# ExcelFile 객체를 사용하여 엑셀 파일 로드
excel_file = pd.ExcelFile(file_path)

# 엑셀 파일 내의 시트 이름 목록 가져오기
sheet_names = excel_file.sheet_names

# 시트가 정확히 두 개인지 확인
if len(sheet_names) == 2:
    # 시트 이름을 사용하여 각 시트를 DataFrame으로 읽기
    df_jsi = pd.read_excel(file_path, sheet_name=sheet_names[0])
    df_susi = pd.read_excel(file_path, sheet_name=sheet_names[1])

    # 수시합불자료 데이터 클리닝
    df_susi.columns = df_susi.iloc[1]  # 두 번째 행을 열 이름으로 설정
    df_susi = df_susi.drop([0, 1])      # 첫 두 행 삭제
    df_susi.reset_index(drop=True, inplace=True) # 인덱스 재설정

    # 정시합불자료 데이터 클리닝
    df_jsi.columns = df_jsi.iloc[1]    # 두 번째 행을 열 이름으로 설정
    df_jsi = df_jsi.drop([0, 1])        # 첫 두 행 삭제
    df_jsi.reset_index(drop=True, inplace=True) # 인덱스 재설정
    df_jsi = df_jsi.rename(columns={'군': '지원\n시기'}) # '군' 열을 '지원시기'로 이름 변경


    # 정시합불자료의 열 이름 변경
    df_jsi.columns.values[32:36] = ['국어_' + col for col in df_jsi.columns[32:36]]
    df_jsi.columns.values[36:41] = ['수학_' + col for col in df_jsi.columns[36:41]]
    df_jsi.columns.values[41:45] = ['영어_' + col for col in df_jsi.columns[41:45]]
    df_jsi.columns.values[45:51] = ['선택1_' + col for col in df_jsi.columns[45:51]]
    df_jsi.columns.values[51:56] = ['선택2_' + col for col in df_jsi.columns[51:56]]
    df_jsi.columns.values[56:60] = ['한국사_' + col for col in df_jsi.columns[56:60]]
    df_jsi.columns.values[60:65] = ['제2외국어_' + col for col in df_jsi.columns[60:65]]


    ##5. 수시합불자료의 열 이름 변경
    df_susi.columns.values[32:36] = ['국어_' + col for col in df_susi.columns[32:36]]
    df_susi.columns.values[36:41] = ['수학_' + col for col in df_susi.columns[36:41]]
    df_susi.columns.values[41:45] = ['영어_' + col for col in df_susi.columns[41:45]]
    df_susi.columns.values[45:51] = ['선택1_' + col for col in df_susi.columns[45:51]]
    df_susi.columns.values[51:56] = ['선택2_' + col for col in df_susi.columns[51:56]]
    df_susi.columns.values[56:60] = ['한국사_' + col for col in df_susi.columns[56:60]]
    df_susi.columns.values[60:65] = ['제2외국어_' + col for col in df_susi.columns[60:65]]


    # 열 이름에서 \n 삭제
    df_jsi.columns = df_jsi.columns.str.replace('\n', '', regex=False)
    df_susi.columns = df_susi.columns.str.replace('\n', '', regex=False)

    # 두 DataFrame을 합치기
    combined_df = pd.concat([df_susi, df_jsi], axis=0)


    # 열 데이터를 숫자형으로 변환
    cols_to_convert = ['국어_백분위', '수학_백분위', '선택1_백분위', '선택2_백분위',
                   '국어_표준점수', '수학_표준점수', '선택1_표준점수', '선택2_표준점수',
                   '국어_등급', '수학_등급', '영어_등급', '선택1_등급', '선택2_등급']

    for col in cols_to_convert:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')


    # 1. '국어_표준점수', '수학_표준점수', '선택1_표준점수', '선택2_표준점수' 열의 합계를 '표점합_수능' 열에 저장
    combined_df['표점합_수능'] = combined_df['국어_표준점수'] + combined_df['수학_표준점수'] + combined_df['선택1_표준점수'] + combined_df['선택2_표준점수']

    # 2. '국어_백분위', '수학_백분위', '선택1_백분위', '선택2_백분위'를 사용하여 '백분위_수능' 열 계산
    combined_df['백분위_수능'] = (combined_df['국어_백분위'] + combined_df['수학_백분위'] + (combined_df['선택1_백분위'] + combined_df['선택2_백분위']) / 2) / 3

    # 3. '국어_등급', '수학_등급', '영어_등급', '선택1_등급', '선택2_등급'을 사용하여 '등급_수능' 열 계산
    combined_df['등급_수능'] = (combined_df['국어_등급'] + combined_df['수학_등급'] + combined_df['영어_등급'] + (combined_df['선택1_등급'] + combined_df['선택2_등급']) / 2) / 3


    # 결과 DataFrame 출력
    print(combined_df.tail())


    # 새로운 경로와 파일명으로 엑셀 파일 저장
    new_path = '/content/drive/MyDrive/python/habbul/'
    combined_df.to_excel(f'{new_path}sinmok2020_수시정시.xlsx', index=False)

else:
    print("합칠 수 있는 시트가 두 개가 아닙니다.")

In [ ]:
# 일부 열을 확인하기: 33rd to 36th
df_jsi.iloc[:, 60:65]

In [ ]:
#특정시트 삭제하기
import pandas as pd

# Excel 파일 경로
file_path = '/mnt/data/habbul_2020sinmok.xlsx'

# 모든 시트를 딕셔너리 형태로 불러오기
excel_data = pd.read_excel(file_path, sheet_name=None)

# '2020합불통계' 시트 삭제
del excel_data['2020합불통계']

# 수정된 내용을 새 파일로 저장
modified_file_path = '/mnt/data/habbul_2020sinmok_modified.xlsx'
with pd.ExcelWriter(modified_file_path) as writer:
    for sheet_name, data in excel_data.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)


In [ ]:
# 2개 시트의 열 이름이 동일한지 확인

import pandas as pd

# 엑셀 파일 경로 설정
path = '/content/drive/MyDrive/python/sinmok/'
file_path = f'{path}habbul_2020sinmok.xlsx'

# ExcelFile 객체를 사용하여 엑셀 파일 로드
excel_file = pd.ExcelFile(file_path)

# 엑셀 파일 내의 시트 이름 목록 가져오기
sheet_names = excel_file.sheet_names

if len(sheet_names) == 2:
    # 시트 이름을 사용하여 각 시트를 DataFrame으로 읽기
    df_jsi = pd.read_excel(file_path, sheet_name=sheet_names[0])
    df_susi = pd.read_excel(file_path, sheet_name=sheet_names[1])



    # 정시합불자료 데이터 클리닝
    df_jsi.columns = df_jsi.iloc[1]    # 두 번째 행을 열 이름으로 설정
    df_jsi = df_jsi.drop([0, 1])        # 첫 두 행 삭제
    df_jsi.reset_index(drop=True, inplace=True) # 인덱스 재설정
    df_jsi = df_jsi.rename(columns={'군': '지원\n시기'}) # '군' 열을 '지원시기'로 이름 변경

    # 수시합불자료 데이터 클리닝
    df_susi.columns = df_susi.iloc[1]  # 두 번째 행을 열 이름으로 설정
    df_susi = df_susi.drop([0, 1])      # 첫 두 행 삭제
    df_susi.reset_index(drop=True, inplace=True) # 인덱스 재설정


    # 정시합불자료의 열 이름 변경
    df_jsi.columns.values[32:36] = ['국어_' + col for col in df_jsi.columns[32:36]]
    df_jsi.columns.values[36:41] = ['수학_' + col for col in df_jsi.columns[36:41]]
    df_jsi.columns.values[41:45] = ['영어_' + col for col in df_jsi.columns[41:45]]
    df_jsi.columns.values[45:51] = ['선택1_' + col for col in df_jsi.columns[45:51]]
    df_jsi.columns.values[51:56] = ['선택2_' + col for col in df_jsi.columns[51:56]]
    df_jsi.columns.values[56:60] = ['한국사_' + col for col in df_jsi.columns[56:60]]
    df_jsi.columns.values[60:65] = ['제2외국어_' + col for col in df_jsi.columns[60:65]]


    ##5. 수시합불자료의 열 이름 변경
    df_susi.columns.values[32:36] = ['국어_' + col for col in df_susi.columns[32:36]]
    df_susi.columns.values[36:41] = ['수학_' + col for col in df_susi.columns[36:41]]
    df_susi.columns.values[41:45] = ['영어_' + col for col in df_susi.columns[41:45]]
    df_susi.columns.values[45:51] = ['선택1_' + col for col in df_susi.columns[45:51]]
    df_susi.columns.values[51:56] = ['선택2_' + col for col in df_susi.columns[51:56]]
    df_susi.columns.values[56:60] = ['한국사_' + col for col in df_susi.columns[56:60]]
    df_susi.columns.values[60:65] = ['제2외국어_' + col for col in df_susi.columns[60:65]]


    # df_susi와 df_jsi의 열 이름 출력
    print("df_susi columns:", df_susi.columns)
    print("df_jsi columns:", df_jsi.columns)

    # 열 이름 비교
    if set(df_susi.columns) == set(df_jsi.columns):
        print("열 이름이 일치합니다.")
    else:
        print("열 이름이 일치하지 않습니다.")
else:
    print("합칠 수 있는 시트가 두 개가 아닙니다.")


In [ ]:
#1개 파일에 1개의 시트(ex: 수시)를 전처리 - 2020sinmok.xlsx으로 테스트

import pandas as pd

# 엑셀 파일 경로 설정
path = '/content/drive/MyDrive/python/sinmok/'
file_path = f'{path}habbul_2020sinmok.xlsx'

# ExcelFile 객체를 사용하여 엑셀 파일 로드
excel_file = pd.ExcelFile(file_path)

# 엑셀 파일 내의 시트 이름 목록 가져오기
sheet_names = excel_file.sheet_names

# 시트가 정확히 두 개인지 확인
if len(sheet_names) == 2:
    # 시트 이름을 사용하여 각 시트를 DataFrame으로 읽기
    df_jsi = pd.read_excel(file_path, sheet_name=sheet_names[0])
    df_susi = pd.read_excel(file_path, sheet_name=sheet_names[1])

    # 수시합불자료 데이터 클리닝
    df_susi.columns = df_susi.iloc[1]  # 두 번째 행을 열 이름으로 설정
    df_susi = df_susi.drop([0, 1])      # 첫 두 행 삭제
    df_susi.reset_index(drop=True, inplace=True) # 인덱스 재설정

    # 정시합불자료 데이터 클리닝
    df_jsi.columns = df_jsi.iloc[1]    # 두 번째 행을 열 이름으로 설정
    df_jsi = df_jsi.drop([0, 1])        # 첫 두 행 삭제
    df_jsi.reset_index(drop=True, inplace=True) # 인덱스 재설정
    df_jsi = df_jsi.rename(columns={'군': '지원\n시기'}) # '군' 열을 '지원시기'로 이름 변경


    # 정시합불자료의 열 이름 변경
    df_jsi.columns.values[32:36] = ['국어_' + col for col in df_jsi.columns[32:36]]
    df_jsi.columns.values[36:41] = ['수학_' + col for col in df_jsi.columns[36:41]]
    df_jsi.columns.values[41:45] = ['영어_' + col for col in df_jsi.columns[41:45]]
    df_jsi.columns.values[45:51] = ['선택1_' + col for col in df_jsi.columns[45:51]]
    df_jsi.columns.values[51:56] = ['선택2_' + col for col in df_jsi.columns[51:56]]
    df_jsi.columns.values[56:60] = ['한국사_' + col for col in df_jsi.columns[56:60]]
    df_jsi.columns.values[60:65] = ['제2외국어_' + col for col in df_jsi.columns[60:65]]


    ##5. 수시합불자료의 열 이름 변경
    df_susi.columns.values[32:36] = ['국어_' + col for col in df_susi.columns[32:36]]
    df_susi.columns.values[36:41] = ['수학_' + col for col in df_susi.columns[36:41]]
    df_susi.columns.values[41:45] = ['영어_' + col for col in df_susi.columns[41:45]]
    df_susi.columns.values[45:51] = ['선택1_' + col for col in df_susi.columns[45:51]]
    df_susi.columns.values[51:56] = ['선택2_' + col for col in df_susi.columns[51:56]]
    df_susi.columns.values[56:60] = ['한국사_' + col for col in df_susi.columns[56:60]]
    df_susi.columns.values[60:65] = ['제2외국어_' + col for col in df_susi.columns[60:65]]


    # 열 이름에서 \n 삭제
    df_jsi.columns = df_jsi.columns.str.replace('\n', '', regex=False)
    df_susi.columns = df_susi.columns.str.replace('\n', '', regex=False)

    # 두 DataFrame을 합치기
    combined_df = pd.concat([df_susi, df_jsi], axis=0)


    # 열 데이터를 숫자형으로 변환
    cols_to_convert = ['국어_백분위', '수학_백분위', '선택1_백분위', '선택2_백분위',
                   '국어_표준점수', '수학_표준점수', '선택1_표준점수', '선택2_표준점수',
                   '국어_등급', '수학_등급', '영어_등급', '선택1_등급', '선택2_등급']

    for col in cols_to_convert:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')


    # 1. '국어_표준점수', '수학_표준점수', '선택1_표준점수', '선택2_표준점수' 열의 합계를 '표점합_수능' 열에 저장
    combined_df['표점합_수능'] = combined_df['국어_표준점수'] + combined_df['수학_표준점수'] + combined_df['선택1_표준점수'] + combined_df['선택2_표준점수']

    # 2. '국어_백분위', '수학_백분위', '선택1_백분위', '선택2_백분위'를 사용하여 '백분위_수능' 열 계산
    combined_df['백분위_수능'] = (combined_df['국어_백분위'] + combined_df['수학_백분위'] + (combined_df['선택1_백분위'] + combined_df['선택2_백분위']) / 2) / 3

    # 3. '국어_등급', '수학_등급', '영어_등급', '선택1_등급', '선택2_등급'을 사용하여 '등급_수능' 열 계산
    combined_df['등급_수능'] = (combined_df['국어_등급'] + combined_df['수학_등급'] + combined_df['영어_등급'] + (combined_df['선택1_등급'] + combined_df['선택2_등급']) / 2) / 3


    # 결과 DataFrame 출력
    print(combined_df.tail())


    # 새로운 경로와 파일명으로 엑셀 파일 저장
    new_path = '/content/drive/MyDrive/python/habbul/'
    combined_df.to_excel(f'{new_path}sinmok2020_수시정시.xlsx', index=False)

else:
    print("합칠 수 있는 시트가 두 개가 아닙니다.")

    # 시트 이름을 사용하여 각 시트를 DataFrame으로 읽기
    #df_jsi = pd.read_excel(file_path, sheet_name=sheet_names[0])
    df_susi = pd.read_excel(file_path, sheet_name=sheet_names[1])

    # 수시합불자료 데이터 클리닝
    df_susi.columns = df_susi.iloc[1]  # 두 번째 행을 열 이름으로 설정
    df_susi = df_susi.drop([0, 1])      # 첫 두 행 삭제
    df_susi.reset_index(drop=True, inplace=True) # 인덱스 재설정

    # 정시합불자료 데이터 클리닝
    #df_jsi.columns = df_jsi.iloc[1]    # 두 번째 행을 열 이름으로 설정
    #df_jsi = df_jsi.drop([0, 1])        # 첫 두 행 삭제
    #df_jsi.reset_index(drop=True, inplace=True) # 인덱스 재설정
    #df_jsi = df_jsi.rename(columns={'군': '지원\n시기'}) # '군' 열을 '지원시기'로 이름 변경


    # 정시합불자료의 열 이름 변경
    #df_jsi.columns.values[32:36] = ['국어_' + col for col in df_jsi.columns[32:36]]
    #df_jsi.columns.values[36:41] = ['수학_' + col for col in df_jsi.columns[36:41]]
    #df_jsi.columns.values[41:45] = ['영어_' + col for col in df_jsi.columns[41:45]]
    #df_jsi.columns.values[45:51] = ['선택1_' + col for col in df_jsi.columns[45:51]]
    #df_jsi.columns.values[51:56] = ['선택2_' + col for col in df_jsi.columns[51:56]]
    #df_jsi.columns.values[56:60] = ['한국사_' + col for col in df_jsi.columns[56:60]]
    #df_jsi.columns.values[60:65] = ['제2외국어_' + col for col in df_jsi.columns[60:65]]


    ##5. 수시합불자료의 열 이름 변경
    df_susi.columns.values[32:36] = ['국어_' + col for col in df_susi.columns[32:36]]
    df_susi.columns.values[36:41] = ['수학_' + col for col in df_susi.columns[36:41]]
    df_susi.columns.values[41:45] = ['영어_' + col for col in df_susi.columns[41:45]]
    df_susi.columns.values[45:51] = ['선택1_' + col for col in df_susi.columns[45:51]]
    df_susi.columns.values[51:56] = ['선택2_' + col for col in df_susi.columns[51:56]]
    df_susi.columns.values[56:60] = ['한국사_' + col for col in df_susi.columns[56:60]]
    df_susi.columns.values[60:65] = ['제2외국어_' + col for col in df_susi.columns[60:65]]


    # 열 이름에서 \n 삭제
    #df_jsi.columns = df_jsi.columns.str.replace('\n', '', regex=False)
    df_susi.columns = df_susi.columns.str.replace('\n', '', regex=False)


    # 열 데이터를 숫자형으로 변환
    cols_to_convert = ['국어_백분위', '수학_백분위', '선택1_백분위', '선택2_백분위',
                   '국어_표준점수', '수학_표준점수', '선택1_표준점수', '선택2_표준점수',
                   '국어_등급', '수학_등급', '영어_등급', '선택1_등급', '선택2_등급']

    for col in cols_to_convert:
        df_susi[col] = pd.to_numeric(df_susi[col], errors='coerce')


    # 1. '국어_표준점수', '수학_표준점수', '선택1_표준점수', '선택2_표준점수' 열의 합계를 '표점합_수능' 열에 저장
    df_susi['표점합_수능'] = df_susi['국어_표준점수'] + df_susi['수학_표준점수'] + df_susi['선택1_표준점수'] + df_susi['선택2_표준점수']

    # 2. '국어_백분위', '수학_백분위', '선택1_백분위', '선택2_백분위'를 사용하여 '백분위_수능' 열 계산
    df_susi['백분위_수능'] = (df_susi['국어_백분위'] + df_susi['수학_백분위'] + (df_susi['선택1_백분위'] + df_susi['선택2_백분위']) / 2) / 3

    # 3. '국어_등급', '수학_등급', '영어_등급', '선택1_등급', '선택2_등급'을 사용하여 '등급_수능' 열 계산
    df_susi['등급_수능'] = (df_susi['국어_등급'] + df_susi['수학_등급'] + df_susi['영어_등급'] + (df_susi['선택1_등급'] + df_susi['선택2_등급']) / 2) / 3


    # 결과 DataFrame 출력
    print(df_susi.tail())


    # 새로운 경로와 파일명으로 엑셀 파일 저장
    new_path = '/content/drive/MyDrive/python/habbul/'
    df_susi.to_excel(f'{new_path}sinmok2020_수시정시.xlsx', index=False)
